# Pricing of Maturity Monitored Barrier Security

This notebook prices the _"NBC Auto Callable Contingent Memory Income Note"_ described in this [link](https://nbcstructuredsolutions.ca/detailProduit.aspx?lequel=5317#documents).  This note is referred to as the _"security"_ for the remainder of this Notebook. 

This will be accomplished by first creating a function to generate the payout of the security if given the return of the underlying asset, which is the _Shares® Core S&P 500 Index ETF (CAD-Hedged)_.   This asset is referred to the _"ETF"_ for the remainder of the Notebook.

With this function defined we estimate the volatility of the ETF and then perform a Monte Carlo simulation of the ETF performance and the resulting security payout.  With the population of payout profiles we can estimate the fair price of the security.

## 1. Create the security payout function

#### Import some necessary modules.

In [73]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

%matplotlib inline

#### Define some variables

In [74]:
barrier = -0.25      # Maturity-monitored barrier
coupon = 5.75 / 2    # Semi-annual coupon payment
call_threshold = 0.1 # Call threshold
face_value = 100     # Face value of the security
pf = 0.0             # Participation Factor
deferred_ppf = 0.05  # Deferred payment participation factor
current_ppf = 0.01   # Current payment participation factor
vrt = 0.0            # Variable return threshold

#### Test the payout function with the examples given in this [link](https://nbcstructuredsolutions.ca/_afficheDocument.aspx?doc=05317_EN_CLIENT_SUMMARY.pdf&secur=0&rnd=0.01401764).

In [84]:
returns = np.array([[ -9, -41, -12, -42,   -46, -48, -46.5, -50,   -47, -45.5, -47,   -43, -49,   -50],
                    [ -5,  -3,   6, -10,   -41, -46, -41,    -9,   -20, -49,   -45.5, -44, -41.5, -15],
                    [ -5, -10,  -7, -13.5, -46, -50, -47,   -46.5, -47, -42,   -44,   -14,  -4,    35],
                    [-10, -14,  -4, -13,   -11,  25,   1,    50,   -40, -60,    -5,     5,  95,     5]]).T/100.0

expected_payout = np.zeros_like(returns)
expected_payout[0, 0] = 2.875
expected_payout[2, 0] = 6.40

expected_payout[[0, 1, 3, 8], 1] = 2.875
expected_payout[2, 1] = 2.935
expected_payout[7, 1] = 12.3
expected_payout[-1, 1] = 14.875

expected_payout[[0, 1, 2, 3, -2], 2] = 2.875
expected_payout[-3, 2] = 23.55
expected_payout[-1, 2] = 3.225

expected_payout[0:5, 3] = 2.875
expected_payout[5, 3] = 3.125

print('Returns:')
print(returns)
print('\nExpected Payout:')
print(expected_payout)


Returns:
[[-0.09  -0.05  -0.05  -0.1  ]
 [-0.41  -0.03  -0.1   -0.14 ]
 [-0.12   0.06  -0.07  -0.04 ]
 [-0.42  -0.1   -0.135 -0.13 ]
 [-0.46  -0.41  -0.46  -0.11 ]
 [-0.48  -0.46  -0.5    0.25 ]
 [-0.465 -0.41  -0.47   0.01 ]
 [-0.5   -0.09  -0.465  0.5  ]
 [-0.47  -0.2   -0.47  -0.4  ]
 [-0.455 -0.49  -0.42  -0.6  ]
 [-0.47  -0.455 -0.44  -0.05 ]
 [-0.43  -0.44  -0.14   0.05 ]
 [-0.49  -0.415 -0.04   0.95 ]
 [-0.5   -0.15   0.35   0.05 ]]

Expected Payout:
[[ 2.875  2.875  2.875  2.875]
 [ 0.     2.875  2.875  2.875]
 [ 6.4    2.935  2.875  2.875]
 [ 0.     2.875  2.875  2.875]
 [ 0.     0.     0.     2.875]
 [ 0.     0.     0.     3.125]
 [ 0.     0.     0.     0.   ]
 [ 0.    12.3    0.     0.   ]
 [ 0.     2.875  0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.    23.55   0.   ]
 [ 0.     0.     2.875  0.   ]
 [ 0.    14.875  3.225  0.   ]]


#### Define the payout function

In [87]:
from itertools import groupby
def payout(returns):
    """
    Args:
        returns (np.array):  [n_dates, n_simulations] The ETF returns.
        
    Returns:
        np.array:  [n_dates, n_simulations]  The resulting payouts.
    """
    n_dates = returns.shape[0]
    n_sims = returns.shape[1]

    # Determine the returns that are above the barrier (hits)
    hits = returns >= b
    miss = np.logical_not(hits)
    
    # Calculate the memory
    memory = np.zeros_like(returns)
    for i in range(1, n_dates):
        memory[i, :] = (memory[i - 1, :] + 1) * miss[i - 1, :]
    
    # Calculate deferred payment variable return if memory > 0
    deferred = (face_value * (returns - barrier) * deferred_ppf) * (memory > 0)
    
    # Calculate Payment Variable Return
    variable = face_value * np.clip(returns - vrt, a_min=0.0, a_max=None) * current_ppf

    # Calculate the total coupon payment
    coupons = hits * (coupon * (memory + 1) + deferred) + variable
    
    # Zero all coupons after the call limit has been achieved
    # Dummy created so argmax returns the size of the array if limit not volated
    dummy = np.vstack((returns, np.ones((1, n_sims)) * (call_threshold + 1)))
    hit_limit = np.argmax(dummy > call_threshold, axis=0) + 1
    for i in range(n_sims):
        coupons[hit_limit[i]:, i] = 0.0
    
    return coupons

delta = np.abs(payout(returns) - expected_payout)
if np.max(delta) > 0:
    print("Error detected in test set.")
    print(delta)
else:
    print("Exact match with test set!!")


Exact match with test set!!
